<a href="https://colab.research.google.com/github/MoBanerjee/SC4001_Project/blob/main/ViT_PromptTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective of Notebook
### This notebook aims to enhance the performance of a Vision Transformer (ViT) model on the Flowers102 dataset by integrating learnable prompt tokens that provide contextual information. We explore various prompt lengths, identify the optimal length, and evaluate the model’s accuracy to demonstrate the effectiveness of visual prompt-based tuning.

In [59]:
# necessary imports
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import ViTModel
from torch.utils.data import DataLoader
from torchvision.datasets import Flowers102

In [60]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize image to 224x224 pixels
    transforms.ToTensor(),  # convert image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # normalize with standard mean and std
])

In [61]:
# load the Flowers102 dataset for training, validation, and testing, applying the defined transformations
train_data = Flowers102(root='./data', split='train', transform=transform, download=True)  # training set
val_data = Flowers102(root='./data', split='val', transform=transform, download=True)  # validation set
test_data = Flowers102(root='./data', split='test', transform=transform, download=True)  # test set

In [62]:
# create DataLoader objects for the training, validation, and test datasets with specified batch sizes
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)  # shuffle training data for randomization
val_loader = DataLoader(val_data, batch_size=16, shuffle=False)  # validation data without shuffling
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)  # test data without shuffling

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # set device

### We designed a Vision Transformer (ViT) model with an additional learnable prompt mechanism to enhance its performance on image classification tasks. By adding a VisualPrompt class, we introduced a set of learnable prompt tokens that are prepended to the image embeddings, providing contextual cues that guide the model’s attention. These tokens, initialized as trainable parameters, are concatenated to the sequence of embeddings for each image batch. We then passed this enhanced sequence through the ViT encoder, allowing it to utilize the prompts during its attention mechanisms. Finally, we extracted the representation of the [CLS] token from the model’s output and fed it through a classifier to predict the image class. This setup aims to leverage the additional information from the prompt tokens for improved classification on the Flowers102 dataset.

In [64]:
class VisualPrompt(nn.Module):
    def __init__(self, prompt_length=5, embed_dim=768):
        # initialize the VisualPrompt class, inheriting from nn.Module
        # prompt_length: number of tokens to be added as learnable prompts
        # embed_dim: dimensionality of each prompt token, which should match the embedding dimension of the input
        super(VisualPrompt, self).__init__()

        # create a set of learnable prompt tokens with the specified length and embedding dimension
        # nn.Parameter ensures these tokens are part of the model's parameters and are updated during training
        self.prompt_tokens = nn.Parameter(torch.randn(prompt_length, embed_dim))  # shape [prompt_length, embed_dim]

    def forward(self, x):
        # define the forward pass for the VisualPrompt layer
        # x: input embedding tensor with dimensions [batch_size, sequence_length, embed_dim]

        batch_size = x.size(0)  # get the batch size from the input tensor
        # expand the prompt tokens to match the batch size, so they can be concatenated to the input
        # unsqueeze(0) adds a batch dimension to the prompt tokens, making the shape [1, prompt_length, embed_dim]
        # expand replicates the prompt tokens across the batch dimension, making the shape [batch_size, prompt_length, embed_dim]
        prompt_tokens = self.prompt_tokens.unsqueeze(0).expand(batch_size, -1, -1)

        # concatenate the expanded prompt tokens with the input tensor along the sequence dimension
        # result shape will be [batch_size, prompt_length + sequence_length, embed_dim]
        return torch.cat((prompt_tokens, x), dim=1)


In [65]:
class ViTWithPrompt(nn.Module):
    def __init__(self, vit_model, prompt_length=5):
        # initialize ViTWithPrompt, inheriting from nn.Module
        # vit_model: the pre-trained Vision Transformer (ViT) model to be used
        # prompt_length: number of learnable prompt tokens to be prepended to the input sequence
        super(ViTWithPrompt, self).__init__()

        self.vit_model = vit_model  # store the provided pre-trained ViT model

        # initialize the VisualPrompt layer with the specified prompt length
        # this layer generates learnable prompt tokens to guide the ViT model
        self.visual_prompt = VisualPrompt(prompt_length=prompt_length)

        # define a classifier layer to map the output of the ViT model to 102 classes
        # the number of output classes is set to 102 for Flowers102 dataset
        # vit_model.config.hidden_size represents the hidden size of the ViT model, ensuring compatibility
        self.classifier = nn.Linear(vit_model.config.hidden_size, 102)

    def forward(self, pixel_values):
        # define the forward pass for the ViTWithPrompt model
        # pixel_values: input tensor representing images in a format suitable for the ViT model

        # pass the input pixel values through the ViT model to get hidden states
        # output_hidden_states=True ensures that hidden states are returned for further processing
        vit_outputs = self.vit_model(pixel_values=pixel_values, output_hidden_states=True)

        # extract the last hidden state, which contains the encoded representations for each token
        # last_hidden_state shape: [batch_size, sequence_length, hidden_size]
        last_hidden_state = vit_outputs.last_hidden_state

        # pass the last hidden state through the VisualPrompt layer
        # this prepends learnable prompt tokens to the input, creating enhanced contextual representations
        # hidden_with_prompt shape: [batch_size, prompt_length + sequence_length, hidden_size]
        hidden_with_prompt = self.visual_prompt(last_hidden_state)

        # pass the combined hidden states (with prompts) through the ViT encoder
        # this generates new representations that incorporate the prompts
        # outputs_with_prompt shape: [batch_size, prompt_length + sequence_length, hidden_size]
        outputs_with_prompt = self.vit_model.encoder(hidden_with_prompt)

        # extract the [CLS] token from the encoded output for classification purposes
        # cls_output shape: [batch_size, hidden_size]
        cls_output = outputs_with_prompt[0][:, 0]

        # pass the [CLS] token through the classifier to obtain the final logits
        # logits shape: [batch_size, 102]
        logits = self.classifier(cls_output)

        return logits


In [66]:
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224')  # load a pre-trained ViT model
vit_model.to(device)  # move the model to the specified device 

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTSdpaAttention(
          (attention): ViTSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUAct

In [67]:
criterion = nn.CrossEntropyLoss()  # define the loss function as cross-entropy for multi-class classification
optimizer = torch.optim.Adam(vit_model.parameters(), lr=2e-5)  # set up the Adam optimizer with a learning rate of 2e-5 

In [68]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        # initialize early stopping parameters
        self.patience = patience  # number of epochs to wait for improvement
        self.verbose = verbose  # whether to print early stopping messages
        self.counter = 0  # count of epochs without improvement
        self.best_score = None  # best score seen so far
        self.early_stop = False  # flag to indicate if training should stop

    def __call__(self, score, epoch):
        # update early stopping logic based on the current score
        if self.best_score is None:
            # set the initial best score
            self.best_score = score
        elif score < self.best_score:
            # increase counter if there's no improvement
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience} at epoch {epoch}')
            if self.counter >= self.patience:
                # stop if counter reaches patience limit
                self.early_stop = True
                print(f'Early stopping at epoch {epoch}. No improvement in validation accuracy for {self.patience} consecutive epochs.')
        else:
            # reset counter if improvement is observed
            self.best_score = score
            self.counter = 0


In [69]:
def train_model(model, train_loader, val_loader, optimizer, criterion, model_save_path, epochs=5, patience=5):
    # initialize early stopping with specified patience and verbosity
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    best_val_accuracy = 0.0  # track the best validation accuracy
    best_model_wts = None  # store the best model weights

    # lists to store loss and accuracy history
    train_loss_history = []
    train_accuracy_history = []
    val_loss_history = []
    val_accuracy_history = []

    # loop over epochs
    for epoch in range(epochs):
        model.train()  # set model to training mode
        running_loss = 0.0  # cumulative loss for the epoch
        correct = 0  # count of correct predictions
        total = 0  # total number of samples

        # iterate over batches in training data
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # move data to device

            optimizer.zero_grad()  # reset gradients
            outputs = model(images)  # forward pass
            loss = criterion(outputs, labels)  # calculate loss
            loss.backward()  # backward pass
            optimizer.step()  # update model parameters

            running_loss += loss.item()  # accumulate loss

            _, predicted = torch.max(outputs, 1)  # get predicted classes
            total += labels.size(0)  # count total samples
            correct += (predicted == labels).sum().item()  # count correct predictions

        train_loss = running_loss / len(train_loader)  # average training loss
        train_accuracy = correct / total  # training accuracy

        # validate the model and calculate validation loss and accuracy
        val_loss, val_accuracy = validate_model(model, val_loader, criterion)

        # save training and validation metrics
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        # print metrics for the epoch
        print(f"Epoch [{epoch + 1}/{epochs}], "
              f"Train Loss: {train_loss:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Loss: {val_loss:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}")

        # update best model weights if validation accuracy improves
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_wts = model.state_dict()

        # check early stopping condition
        early_stopping(val_accuracy, epoch + 1)
        if early_stopping.early_stop:
            break  # stop training if no improvement in validation accuracy

    # load the best model weights if available
    if best_model_wts is not None:
        model.load_state_dict(best_model_wts)

    # save the model if a save path is provided
    if model_save_path != "None":
        torch.save(model.state_dict(), model_save_path)

    # return loss and accuracy history
    return train_loss_history, train_accuracy_history, val_loss_history, val_accuracy_history


In [70]:
def validate_model(model, val_loader, criterion):
    model.eval()  # set model to evaluation mode
    total_loss = 0.0  # cumulative validation loss
    correct = 0  # count of correct predictions
    total = 0  # total number of samples

    # disable gradient computation for validation
    with torch.no_grad():
        # iterate over validation data batches
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # move data to device
            outputs = model(images)  # forward pass
            loss = criterion(outputs, labels)  # calculate loss

            total_loss += loss.item()  # accumulate loss

            # get predicted classes
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)  # update total samples count
            correct += (predicted == labels).sum().item()  # count correct predictions

    val_loss = total_loss / len(val_loader)  # average validation loss
    val_accuracy = correct / total  # validation accuracy
    return val_loss, val_accuracy  # return loss and accuracy


In [78]:
def test_model(model, test_loader):
    model.eval()  # set model to evaluation mode
    correct = 0  # count of correct predictions
    total = 0  # total number of samples

    # disable gradient computation for testing
    with torch.no_grad():
        # iterate over test data batches
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # move data to device
            outputs = model(images)  # forward pass
            _, predicted = torch.max(outputs, 1)  # get predicted classes
            total += labels.size(0)  # update total samples count
            correct += (predicted == labels).sum().item()  # count correct predictions

    # print test accuracy as a percentage
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [72]:
# list of prompt lengths to experiment with
prompt_lengths = [5, 10, 15, 25, 50, 75, 100, 500, 1000]

# variables to store the best prompt length and highest validation accuracy observed
best_prompt_length = 0  # initialize best prompt length to zero
best_val_accuracy = 0.0  # initialize best validation accuracy to zero

# dictionary to store training and validation history for each prompt length
history_dict = {}

# loop over each prompt length in the list to train and evaluate the model
for prompt_length in prompt_lengths:
    print(f"\nTraining with prompt length: {prompt_length}")  # print current prompt length being used

    # create a new instance of the ViTWithPrompt model with the specified prompt length
    model = ViTWithPrompt(vit_model, prompt_length=prompt_length)
    model.to(device)  # move the model to the device (gpu or cpu)

    # train the model and collect the training and validation loss and accuracy history
    train_loss_history, train_accuracy_history, val_loss_history, val_accuracy_history = train_model(
        model, train_loader, val_loader, optimizer, criterion, epochs=25, model_save_path='None'
    )

    # store the training and validation history in the dictionary using the prompt length as the key
    history_dict[prompt_length] = {
        "train_loss": train_loss_history,  # store list of training loss values per epoch
        "train_accuracy": train_accuracy_history,  # store list of training accuracy values per epoch
        "val_loss": val_loss_history,  # store list of validation loss values per epoch
        "val_accuracy": val_accuracy_history  # store list of validation accuracy values per epoch
    }

    # perform validation on the model after training to obtain final validation loss and accuracy
    val_loss, val_accuracy = validate_model(model, val_loader, criterion)

    # check if the current prompt length yields a higher validation accuracy than previous runs
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy  # update best validation accuracy
        best_prompt_length = prompt_length  # update best prompt length

# print the prompt length that achieved the best validation accuracy after all runs
print(f"\nBest prompt length: {best_prompt_length} with validation accuracy: {best_val_accuracy:.2f}%")


Training with prompt length: 5
Epoch [1/25], Train Loss: 5.8908, Train Accuracy: 0.0539, Validation Loss: 3.4996, Validation Accuracy: 0.2392
Epoch [2/25], Train Loss: 2.0865, Train Accuracy: 0.5373, Validation Loss: 1.7817, Validation Accuracy: 0.6010
Epoch [3/25], Train Loss: 0.5381, Train Accuracy: 0.9020, Validation Loss: 1.1293, Validation Accuracy: 0.7382
Epoch [4/25], Train Loss: 0.0932, Train Accuracy: 0.9941, Validation Loss: 0.9375, Validation Accuracy: 0.7853
Epoch [5/25], Train Loss: 0.0229, Train Accuracy: 1.0000, Validation Loss: 0.8832, Validation Accuracy: 0.7833
EarlyStopping counter: 1 out of 5 at epoch 5
Epoch [6/25], Train Loss: 0.0106, Train Accuracy: 1.0000, Validation Loss: 0.8558, Validation Accuracy: 0.7912
Epoch [7/25], Train Loss: 0.0074, Train Accuracy: 1.0000, Validation Loss: 0.8443, Validation Accuracy: 0.7931
Epoch [8/25], Train Loss: 0.0058, Train Accuracy: 1.0000, Validation Loss: 0.8355, Validation Accuracy: 0.7971
Epoch [9/25], Train Loss: 0.0047, T

In [77]:
# cell for plotting training and validation metrics for different prompt lengths
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np


cmap = cm.get_cmap('tab10', len(history_dict))


plt.figure(figsize=(14, 16))


plt.subplot(4, 1, 1)
for i, (prompt_length, history) in enumerate(history_dict.items()):
    epochs = range(1, len(history["train_loss"]) + 1)
    color = cmap(i)
    plt.plot(epochs, history["train_loss"], label=f'Prompt {prompt_length}', color=color, linewidth=2.5)
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.title('Training Loss for All Prompt Lengths')
plt.grid()


plt.subplot(4, 1, 2)
for i, (prompt_length, history) in enumerate(history_dict.items()):
    epochs = range(1, len(history["val_loss"]) + 1)
    color = cmap(i)
    plt.plot(epochs, history["val_loss"], label=f'Prompt {prompt_length}', color=color, linestyle='--', linewidth=2.5)
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.title('Validation Loss for All Prompt Lengths')
plt.grid()


plt.subplot(4, 1, 3)
for i, (prompt_length, history) in enumerate(history_dict.items()):
    epochs = range(1, len(history["train_accuracy"]) + 1)
    color = cmap(i)
    plt.plot(epochs, history["train_accuracy"], label=f'Prompt {prompt_length}', color=color, linewidth=2.5)
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.title('Training Accuracy for All Prompt Lengths')
plt.grid()


plt.subplot(4, 1, 4)
for i, (prompt_length, history) in enumerate(history_dict.items()):
    epochs = range(1, len(history["val_accuracy"]) + 1)
    color = cmap(i)
    plt.plot(epochs, history["val_accuracy"], label=f'Prompt {prompt_length}', color=color, linestyle='--', linewidth=2.5)
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy for All Prompt Lengths')
plt.grid()


plt.subplots_adjust(bottom=0.15)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=5, fontsize='small')


plt.tight_layout()
plt.show()


<ipython-input-77-9af5f900e13d>:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab10', len(history_dict))


## Analysis of Variation with Prompt Lengths
### The training and validation loss graphs show a rapid decrease in loss across all prompt lengths within the first few epochs, with minimal differences as training progresses. This suggests that the model reaches convergence quickly regardless of prompt length, particularly in terms of training performance.

### However, in the validation accuracy graph, shorter prompt lengths (e.g., 5 and 10) initially lag behind in performance, but as the epochs progress, prompt lengths from 50 to 500 achieve consistently high validation accuracy. Notably, the prompt length of 500 (highest validation accuracy of 0.96) maintains the highest validation accuracy by the end of training, outperforming both shorter and longer prompts.

### This indicates that while many prompt lengths allow the model to perform well, the prompt length of 500 provides an optimal balance, achieving the best validation accuracy without the diminishing returns observed at excessively high prompt lengths, such as 1000.

In [74]:
# print the chosen best prompt length for training
print(f"\nTraining with the best prompt length: {best_prompt_length}")

# initialize the model with the optimal prompt length
best_model = ViTWithPrompt(vit_model, prompt_length=best_prompt_length)

# move the model to the specified device 
best_model.to(device)

# retrain the model and save training and validation history
train_loss_history, train_accuracy_history, val_loss_history, val_accuracy_history = train_model(best_model, train_loader, val_loader, optimizer, criterion, epochs=25, model_save_path='Best_ViT_PromptTuning.pth')


Training with the best prompt length: 500
Epoch [1/25], Train Loss: 1.4898, Train Accuracy: 0.7627, Validation Loss: 0.5065, Validation Accuracy: 0.9020
Epoch [2/25], Train Loss: 0.1046, Train Accuracy: 0.9794, Validation Loss: 0.3048, Validation Accuracy: 0.9431
Epoch [3/25], Train Loss: 0.0122, Train Accuracy: 0.9980, Validation Loss: 0.2658, Validation Accuracy: 0.9500
Epoch [4/25], Train Loss: 0.0011, Train Accuracy: 1.0000, Validation Loss: 0.2560, Validation Accuracy: 0.9539
Epoch [5/25], Train Loss: 0.0007, Train Accuracy: 1.0000, Validation Loss: 0.2506, Validation Accuracy: 0.9559
Epoch [6/25], Train Loss: 0.0005, Train Accuracy: 1.0000, Validation Loss: 0.2473, Validation Accuracy: 0.9549
EarlyStopping counter: 1 out of 5 at epoch 6
Epoch [7/25], Train Loss: 0.0004, Train Accuracy: 1.0000, Validation Loss: 0.2449, Validation Accuracy: 0.9539
EarlyStopping counter: 2 out of 5 at epoch 7
Epoch [8/25], Train Loss: 0.0004, Train Accuracy: 1.0000, Validation Loss: 0.2432, Validat

In [75]:
# cell for plotting training and validation metrics
epochs = range(1, len(train_loss_history) + 1)

plt.figure(figsize=(10, 5))
plt.subplot(2, 1, 1)
plt.plot(epochs, train_loss_history, 'b-', label='Training Loss')
plt.plot(epochs, val_loss_history, 'orange', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss History')
plt.legend()
plt.grid()


plt.subplot(2, 1, 2)
plt.plot(epochs, train_accuracy_history, 'b-', label='Training Accuracy')
plt.plot(epochs, val_accuracy_history, 'orange', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy History')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

In [80]:
# evaluate best model saved earlier on test dataset
test_model(best_model, test_loader)

Test Accuracy: 95.30%


## Analysis of Results
### The training and validation loss graphs show rapid convergence within the first few epochs, with both training and validation loss stabilizing at low levels, indicating effective learning and minimal overfitting. Similarly, the accuracy plot reflects high and stable performance, with training accuracy reaching near-perfect levels and validation accuracy following closely. This suggests that the model generalizes well, with minimal performance gap between training and validation sets. The test accuracy of 95.3% further confirms the model’s strong generalization capability.

# Notably, this was our best performing model among all the models we tried out!